## OpenAI Function Call to Semantically Search Words in the Quran

Yesterday, OpenAI announced a new feature that allows descriptive functions to query GPT. This non-deterministic paradigm opens up many new dimensions for software developers. In this notebook, I share how to use the API.

In the following lines, we use semantic search to find word matches in the Quran. Instead of quering a relational database, we simply ask GPT using a function call. Function calls are different from prompting as we can control the returned values and also preserve tokens as no exhuberant or uncessary content is returned other than that which we have requested. 

Notice, the following search for the word "orphan" returns verses that contain the semantically similar words that closely match it. The search, however, is not always reliable with the current LLM model. I expect it will improve with GPT 4 and beyond.

In [247]:
question = "Where in the Quran is the word 'orphan' found?"

In [37]:
#!pip install termcolor

In [39]:
#!pip install tenacity

In [248]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import requests
import json

In [249]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="keys/.env")

openai.api_key = os.getenv('OPEN_API_KEY_FOR_ASK_QURAN')

In [250]:
GPT_MODEL = "gpt-3.5-turbo-0613"

In [251]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [252]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

In [253]:
functions = [
    {
        "name": "get_surah_name",
        "description": "Get name of the Surah when user provides surah number",
        "parameters": {
            "type": "object",
            "properties": {
                "surah_name": {
                    "type": "string",
                    "description": "Name of the Surah",
                }
            },
            "required": ["surah_number"],
        },
    },
    {
        "name": "find_word_in_the_quran",
        "description": "Find all occurrences of Surahs and Ayahs where this word appears in the Quran.",
        "parameters": {
            "type": "object",
            "properties": {
                "ayahs": {                    
                    "type": "object",
                    "description": "A list of ayahs containing this word in the Quran",
                    "properties": {
                        "surah_number": {
                            "type": "integer",
                            "description": "The surah number where this word appears.",
                        },
                        "ayah_number": {
                            "type": "integer",
                            "description": "The ayah number where this word appears.",
                        },                
                        "surah_name": {
                            "type": "string",
                            "description": "Name of the Surah where this word appears.",
                        },
                        "ayah_content_in_arabic": {
                            "type": "string",
                            "description": "Entire content of the ayah in Arabic that contains this word.",
                        },   
                        "ayah_content_in_english": {
                            "type": "string",
                            "description": "Entire content of the ayah in English that contains this word.",
                        },                         
                    }
                },
                "source": {
                    "type": "string",
                    "description": "Source where this information was gathered.",
                },                
            },
            "required": ["surah_number", "ayah_number"],
        },
    }         
]

In [254]:
conversation = Conversation()

In [255]:
conversation.add_message("user", question)

In [256]:
chat_response = chat_completion_request(
    conversation.conversation_history,
    functions = functions
)

In [259]:
json.loads(chat_response.json()['choices'][0]['message']['function_call']['arguments'])

{'ayahs': [{'surah_number': 2,
   'ayah_number': 220,
   'surah_name': 'Al-Baqarah',
   'ayah_content_in_arabic': 'وَيَسْأَلُونَكَ عَنِ الْيَتَامَىٰ ۖ قُلْ إِصْلَاحٌ لَّهُمْ خَيْرٌ ۖ وَإِن تُخَالِطُوهُمْ فَإِخْوَانُكُمْ ۚ وَٱللَّهُ يَعْلَمُ ٱلْمُفْسِدَ مِنَ ٱلْمُصْلِحِ ۚ وَلَوْ شَاءَ ٱللَّهُ لَأَعْنَتَكُمْ ۚ إِنَّ ٱللَّهَ عَزِيزٌ حَكِيمٌ',
   'ayah_content_in_english': 'And they ask you about orphans. Say, "Improvement for them is best. And if you mix your affairs with theirs - they are your brothers. And Allah knows the corrupter from the amender. And if Allah had willed, He could have put you in difficulty. Indeed, Allah is Exalted in Might and Wise."'},
  {'surah_number': 93,
   'ayah_number': 9,
   'surah_name': 'Ad-Duha',
   'ayah_content_in_arabic': 'أَمَّا ٱلْيَتِيمَ فَلَا تَقْهَرْ',
   'ayah_content_in_english': 'But as for the orphan, do not oppress [him].'},
  {'surah_number': 93,
   'ayah_number': 10,
   'surah_name': 'Ad-Duha',
   'ayah_content_in_arabic': 'وَأَمَّا ٱلسَّآ